In [6]:
import sqlite3
!pip install qrcode
import qrcode
import shutil
import pandas as pd
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive, files
from pathlib import Path
import os
import time
import threading

# Mount Google Drive for storing bills
drive.mount('/content/drive')

# Define paths for storing bills and UTR codes
csv_file_path = "/products.csv"
BILL_FOLDER = "/content/drive/My Drive/SmartCart_Bills/"
Path(BILL_FOLDER).mkdir(parents=True, exist_ok=True)
BILL_PATH = os.path.join(BILL_FOLDER, "bill.txt")
UTR_CSV_PATH = "/content/UTR.csv"

print(f"✅ Bill folder set to: {BILL_FOLDER}")

# Create SQLite database for products
def create_database():
    conn = sqlite3.connect('store.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS products (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        barcode TEXT UNIQUE,
                        name TEXT,
                        price REAL,
                        stock INTEGER,
                        category TEXT,
                        discount REAL)''')
    if Path(csv_file_path).exists():
        df = pd.read_csv(csv_file_path, dtype={'barcode': str})
        for _, row in df.iterrows():
            cursor.execute(
                "INSERT OR IGNORE INTO products (barcode, name, price, stock, category, discount) VALUES (?, ?, ?, ?, ?, ?)",
                (row['barcode'], row['name'], row['price'], row['stock'], row['category'], row['discount'])
            )
    conn.commit()
    conn.close()

create_database()

# Function to get product details
def get_product(barcode):
    conn = sqlite3.connect('store.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name, price, discount FROM products WHERE barcode=?", (barcode,))
    product = cursor.fetchone()
    conn.close()
    return product if product else ("Unknown Product", 0.0, 0.0)

# Shopping Cart
cart = {}

# Function to add item to cart
def add_to_cart(b):
    barcode = barcode_input.value.strip()
    quantity = quantity_input.value
    barcode_input.value = ""  # Clear input field

    if not barcode:
        with cart_output:
            cart_output.clear_output()
            print("⚠️ Error: Please enter a barcode!")
        return

    if quantity <= 0:
        with cart_output:
            cart_output.clear_output()
            print("⚠️ Error: Quantity must be at least 1!")
        return

    name, price, discount = get_product(barcode)
    if name == "Unknown Product":
        with cart_output:
            cart_output.clear_output()
            print("⚠️ Error: Product not found!")
        return

    final_price = price - discount
    if barcode in cart:
        cart[barcode]["quantity"] += quantity
    else:
        cart[barcode] = {"name": name, "price": final_price, "quantity": quantity}

    update_cart_display()

# Function to display cart contents
def update_cart_display():
    cart_output.clear_output()
    total = 0
    with cart_output:
        for item in cart.values():
            print(f"{item['name']} - {item['quantity']} x ₹{item['price']} = ₹{item['quantity'] * item['price']}")
            total += item['quantity'] * item['price']
    total_output.clear_output()
    with total_output:
        print(f"💰 Total: ₹{total}")

# Function to save bill
def save_bill_to_file():
    total = sum(item["price"] * item["quantity"] for item in cart.values())
    with open(BILL_PATH, "w") as file:
        file.write("🛒 Smart Cart - Shopping Bill\n")
        file.write("=" * 30 + "\n")
        for item in cart.values():
            file.write(f"{item['name']} - {item['quantity']} x ₹{item['price']} = ₹{item['quantity'] * item['price']}\n")
        file.write("=" * 30 + f"\nTotal: ₹{total}\n")
        file.write("📢 Thank you for shopping with us! 😊\n")

# Function to generate payment QR code
def generate_qr(b):
    total = sum(item["price"] * item["quantity"] for item in cart.values())
    if total == 0:
        with qr_output:
            qr_output.clear_output()
            print("⚠️ Error: Cart is empty!")
        return

    qr_data = f"upi://pay?pa=7014508909@paytm&pn=SmartCart&am={total}&cu=INR"
    qr = qrcode.make(qr_data)
    qr_path = os.path.join(BILL_FOLDER, "payment_qr.png")
    qr.save(qr_path)
    img = Image.open(qr_path).resize((200, 200))
    with qr_output:
        qr_output.clear_output()
        display(img)
    save_bill_to_file()

# Function to verify UTR
failed_attempts = 0
def verify_utr(b):
    global failed_attempts
    utr_code = utr_input.value.strip()

    if not utr_code:
        with feedback_output:
            feedback_output.clear_output()
            print("⚠️ Error: Enter a valid UTR code!")
        return

    try:
        df = pd.read_csv(UTR_CSV_PATH, dtype=str)
        if utr_code in df["UTR"].values:
            with feedback_output:
                feedback_output.clear_output()
                print("✅ Payment Verified! You can now download your bill.")
            download_button.layout.display = 'inline-block'  # Show Download Button
        else:
            failed_attempts += 1
            if failed_attempts >= 3:
                with feedback_output:
                    feedback_output.clear_output()
                    print("❌ Too many failed attempts! You are blocked.")
                verify_utr_button.disabled = True
            else:
                with feedback_output:
                    feedback_output.clear_output()
                    print(f"⚠️ Invalid UTR. Attempts left: {3 - failed_attempts}")
    except Exception as e:
        with feedback_output:
            feedback_output.clear_output()
            print(f"⚠️ Error verifying UTR: {str(e)}")

# Feedback System
selected_rating = 0
def set_rating(n):
    global selected_rating
    selected_rating = n
    for i in range(5):
        star_buttons[i].icon = "star" if i < n else "star-outline"

star_buttons = [widgets.Button(icon="star-outline", layout=widgets.Layout(width="30px")) for _ in range(5)]
for i, btn in enumerate(star_buttons):
    btn.on_click(lambda b, i=i: set_rating(i + 1))

feedback_text = widgets.Textarea(placeholder="Optional feedback...")
submit_button = widgets.Button(description="Submit Feedback", button_style="success")
feedback_output = widgets.Output()

def submit_feedback(b):
    with feedback_output:
        feedback_output.clear_output()
        print("✅ Thanks for your feedback!")

submit_button.on_click(submit_feedback)
stars_box = widgets.HBox(star_buttons)
feedback_section = widgets.VBox([stars_box, feedback_text, submit_button, feedback_output])

# Widgets
barcode_input = widgets.Text(placeholder='Enter Barcode')
quantity_input = widgets.IntSlider(min=1, max=10, description="Quantity:")
add_button = widgets.Button(description="Add to Cart", button_style='primary')
pay_button = widgets.Button(description="Generate Payment QR", button_style='success')
utr_input = widgets.Text(placeholder='Enter UTR Code')
verify_utr_button = widgets.Button(description="Verify UTR", button_style='success')
download_button = widgets.Button(description="Download Bill", button_style='info')
download_button.layout.display = 'none'
progress_bar = widgets.IntProgress(min=0, max=100)

cart_output = widgets.Output()
total_output = widgets.Output()
qr_output = widgets.Output()

# Button Actions
add_button.on_click(add_to_cart)
pay_button.on_click(generate_qr)
verify_utr_button.on_click(verify_utr)
download_button.on_click(lambda b: files.download(BILL_PATH))

# Display UI
display(barcode_input, quantity_input, add_button, cart_output, total_output, pay_button, qr_output, utr_input, verify_utr_button, progress_bar, download_button, feedback_section)


Mounted at /content/drive
✅ Bill folder set to: /content/drive/My Drive/SmartCart_Bills/


Text(value='', placeholder='Enter Barcode')

IntSlider(value=1, description='Quantity:', max=10, min=1)

Button(button_style='primary', description='Add to Cart', style=ButtonStyle())

Output()

...

Output()

Text(value='', placeholder='Enter UTR Code')

Button(button_style='success', description='Verify UTR', style=ButtonStyle())

IntProgress(value=0)

Button(button_style='info', description='Download Bill', layout=Layout(display='none'), style=ButtonStyle())